# Generating LLM Perspectives from GlobalOpinionQA Dataset
The paper [Towards Measuring the Representation of Subjective Global Opinions in Language Models](https://huggingface.co/datasets/Anthropic/llm_global_opinions) introduces the GlobalOpinionQA dataset containing survey questions about global issues and opinions from participants from many countries.

To convert the ordinal multiple choice answers into free text answers, we will use LLMs to generate perspectives based on the aggregate statistics in the dataset and save as a new dataset.

In [ ]:
from dotenv import load_dotenv
import pandas as pd, numpy as np, os

# Load environment variables
load_dotenv()
TEMP_PATH = os.getenv('TEMP_PATH')
DATA_PATH = os.getenv('DATA_PATH')

In [ ]:
from datasets import load_dataset

ds = load_dataset("Anthropic/llm_global_opinions")
df = ds['train'].to_pandas()
df.shape

In [ ]:
df.head()

In [ ]:
import ast

# Convert string representation of defaultdict to regular dict
def convert_defaultdict_str(s):
    try:
        # Extract the dictionary part from the defaultdict string
        dict_str = s.split('defaultdict(<class \'list\'>, ')[1][:-1]
        # Parse the dictionary string
        return ast.literal_eval(dict_str)
    except:
        return {}

df['selections'] = df['selections'].apply(convert_defaultdict_str)
df['options'] = df['options'].apply(ast.literal_eval)


In [ ]:
df.head()


## Structured Generation of LLM Perspectives

In [ ]:
from pydantic import BaseModel
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))


In [ ]:
class CountryPerspective(BaseModel):
    option: str
    perspective: str

class CountryPerspectiveList(BaseModel):
    perspectives: list[CountryPerspective]


def generate_country_perspectives(question: str, options: list[str], country: str, max_retries: int = 1):
    """
    Generate perspectives for the given country explaining their choice of options.
    """
    system_prompt = """You are a helpful assistant that generates multiple perspectives of answers to a question. You will be given a question, a country, and a list of answer options, and you will generate a list of possible answer perspectives from the perspective of people from that country. Make sure you cover all possible perspectives but do not repeat yourself.
    """

    prompt = f"""Question: {question}
    Country: {country}
    Options: {', '.join(options)}
    Now, step by step, outline each broad answer perspective to this question from the perspective of a person from the country and each of the answer options.
    """
    retries = 0
    while retries < max_retries:
        try:
            chat_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=1,
                response_format={
                    'type': 'json_schema',
                    'json_schema': {
                        "name": "CountryPerspectiveList",
                        "schema": CountryPerspectiveList.model_json_schema()
                    }
                }
            )

            result_object = json.loads(chat_response.choices[0].message.content)
            # Validate the response using Pydantic
            validated_response = CountryPerspectiveList.model_validate(result_object)
            return [p.perspective for p in validated_response.perspectives]
        except Exception as e:
            retries += 1
            if retries == max_retries:
                print(f"Error for {country} - {question} (after {retries} retries): {str(e)}")
                return [f"Error generating perspective: {str(e)}"]
            print(f"Attempt {retries} failed, retrying...")
            

In [ ]:
def generate_perspectives(df, total_length=None):
    if total_length is None or total_length > len(df):
        total_length = len(df)
    
    # Create a new dataframe with just the rows we need
    working_df = df.head(total_length).copy()
    
    perspectives = []
    for _, row in working_df.iterrows():
        countries = list(row['selections'].keys())
        options = [str(x) for x in row['options']],
        country_perspectives = {}
        for country in countries:
            country_perspectives[country] = generate_country_perspectives(
                row['question'],
                options, 
                country
            )
        perspectives.append(country_perspectives)
    
    # Only modify the rows we processed
    working_df['country_perspectives'] = perspectives
    return working_df

This is soooOOOooo slooOOoowWWww. Lets get this parallelized!

In [ ]:
from concurrent.futures import ThreadPoolExecutor
# Parallel processing with single progress bar
from tqdm.auto import tqdm

# Split the dataframe into chunks
chunk_size = 15 
total_rows = 150
chunks = np.array_split(df.head(total_rows), 10)

# Function to process each chunk
def process_chunk(chunk_df):
    return generate_perspectives(chunk_df, total_length=len(chunk_df))


# Process chunks in parallel using ThreadPoolExecutor with a single progress bar
with ThreadPoolExecutor(max_workers=10) as executor:
    perspectives_chunks = list(tqdm(
        executor.map(process_chunk, chunks),
        total=len(chunks),
        desc="Processing chunks"
    ))

# Combine results
df_with_perspectives = pd.concat(perspectives_chunks, ignore_index=True)


In [ ]:
# Sample 5 questions and display their details
sample_df = df_with_perspectives.head()

# Display the results in a readable format by printing the question and each of the country perspectives
for index, row in sample_df.iterrows():
    print(row['question'])
    for country, perspectives in row['country_perspectives'].items():
        print(f"\t{country}: {'; '.join(perspectives)}")


In [ ]:
# To keep things separate and clean, we're going to save these to a different file.
df_with_perspectives.to_csv(DATA_PATH+f'GlobalOpinionQA_with_LLM_generated_perspectives_{total_rows}.csv', index=False) 